To perform an exploratory data analysis on the hotel booking data. We will work through the following steps:

- Load the data
- Understand the variables in the data
- Clean the data if necessary
- Analyze the data using descriptive statistics, visualizations, and other techniques

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv("../hotel_booking.csv")

In [ ]:
df.head()

In [ ]:
# check the basic information of the data
df.info()

In [ ]:
#describe the data
df.describe()

Perform data cleaning for the data

In [ ]:
# Fill missing values in the `children` column with 0.
df['children'].fillna(0, inplace=True)

In [ ]:
# Fill missing values in the `country` column with "Unknown".
df['country'].fillna('Unknown', inplace=True)

In [ ]:
# Drop the `company` column since it has too many missing values and is not relevant to our analysis.
# Drop the `agent` column since it has a large number of missing values and is not relevant to our analysis.
df.drop(['company', 'agent'], axis=1, inplace=True)

In [ ]:
# Check for duplicate records
duplicate_rows = df[df.duplicated()]
print("Duplicate Rows:")
print(duplicate_rows)

In [ ]:
#Check for specific duplicate records
# Columns relevant for identifying duplicate bookings
duplicate_columns = [
    'hotel', 'name', 'email', 'phone-number', 'credit_card', 'arrival_date_year', 'arrival_date_month',
    'arrival_date_week_number', 'arrival_date_day_of_month', 'reservation_status_date'
]

# Check for duplicate rows based on the subset of columns
duplicate_rows = df[df.duplicated(subset=duplicate_columns)]
print(duplicate_rows)

To further refine the identification of duplicate bookings, we include the "duplicate_columns".These columns can help differentiate bookings that may have the same contact information but differ in terms of arrival and reservation status dates.

In [ ]:
# Drop the rows where `adults` and `children` are both 0, as these are likely to be invalid bookings.
df = df.loc[(df['adults'] + df['children']) > 0]

In [ ]:
# Create a new column `total_guests` by summing the `adults`, `children`, and `babies` columns.
df['total_guests'] = df['adults'] + df['children'] + df['babies']

We include the total number of guests because different hotels charge differenty depending on amounts of guests.By including babies in the total number of guests, we can accurately calculate the total cost of the booking and analyze any trends or patterns related to the number of guests staying in a room.

In [ ]:
# Drop the `babies` column since it is not relevant to our analysis.
df.drop('babies', axis=1, inplace=True)

We choose to exclude the `babies` column since it is not relevant to our analysis. The number of babies staying in a room may not have a significant impact on the total cost of the booking.

In [ ]:
# Verify the updated DataFrame
print("Updated DataFrame:")
df.head()

In [ ]:
#check data information
df.describe()

Next step is to convert the following coloumns into the appropriate data types:
- arrival_date_month: should be categorical
- country: should be categorical
- meal: should be categorical
- market_segment: should be categorical
- distribution_channel: should be categorical
- reserved_room_type: should be categorical
- assigned_room_type: should be categorical
- deposit_type: should be categorical
- customer_type: should be categorical
- reservation_status: should be categorical
- reservation_status_date: should be datetime

In [ ]:
# Convert categorical columns to category type
df['arrival_date_month'] = pd.Categorical(df['arrival_date_month'])
df['country'] = pd.Categorical(df['country'])
df['meal'] = pd.Categorical(df['meal'])
df['market_segment'] = pd.Categorical(df['market_segment'])
df['distribution_channel'] = pd.Categorical(df['distribution_channel'])
df['reserved_room_type'] = pd.Categorical(df['reserved_room_type'])
df['assigned_room_type'] = pd.Categorical(df['assigned_room_type'])
df['deposit_type'] = pd.Categorical(df['deposit_type'])
df['customer_type'] = pd.Categorical(df['customer_type'])
df['reservation_status'] = pd.Categorical(df['reservation_status'])

In [ ]:
# Convert reservation_status_date to datetime
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

In [ ]:
#check data types again
df.dtypes

In [ ]:
# Drop irrelevant columns for our analysis
df.drop(["name", "email", "phone-number", "credit_card"], axis=1, inplace=True)

In [ ]:
# Remove negative values from "adr" column
df = df[df["adr"] > 0]

In [ ]:
#Create a histogram of lead time to see the distribution of booking lead times
fig1 = px.histogram(df, x="lead_time", nbins=50, title="Distribution of Booking Lead Time")
fig1.show()

In [ ]:
#Histogram: ADR distribution, faceted by arrival month
fig2 = px.histogram(df,
    x='adr',
    color='arrival_date_month',
    nbins=50,
    facet_col='arrival_date_month',
    facet_col_wrap=4,
    title='ADR Distribution by Arrival Month',
    labels={'adr': 'Average Daily Rate (ADR)'}
)
fig2.show()

In [ ]:
#Histogram: Lead Time distribution, faceted by hotel type
fig3 = px.histogram(df,
    x='lead_time',
    color='hotel',
    nbins=50,
    facet_col='hotel',
    title='Lead Time Distribution by Hotel Type',
    labels={'lead_time': 'Lead Time'}
)
fig3.show()

In [ ]:
# Scatter plot: Lead Time vs. ADR
fig4 = px.scatter(df,
    x='lead_time',
    y='adr',
    title='Lead Time vs. ADR',
    labels={'lead_time': 'Lead Time', 'adr': 'ADR'}
)
fig4.show()

In [ ]:
# Scatter plot: Total Guests vs. Stays in Week Nights
fig5 = px.scatter(df,
    x='total_guests',
    y='stays_in_week_nights',
    title='Total Guests vs. Stays in Week Nights',
    labels={'total_guests': 'Total Guests', 'stays_in_week_nights': 'Stays in Week Nights'}
)
fig5.show()

In [ ]:
#Scatter plot: Previous Cancellations vs. Previous Bookings Not Canceled
fig6 = go.Figure()

fig6.add_trace(go.Scatter(
    x=df['previous_cancellations'],
    y=df['previous_bookings_not_canceled'],
    mode='markers',
    marker=dict(color='purple', size=5),
    name='Data',
))

fig6.update_layout(title='Previous Cancellations vs. Previous Bookings Not Canceled', xaxis_title='Previous Cancellations', yaxis_title='Previous Bookings Not Canceled')
fig6.show()